# Load Head with id2label

In [1]:
!pip install -Uq adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00


First, we create the model and load the trained adapter and head. With the head, the labels, and the i2label mapping is automatically loaded. With `model.get_labels()` and `model.get_labels_dict()` you can access them.

In [2]:
from adapters import AutoAdapterModel
from transformers import AutoTokenizer
import numpy as np

model_name = "bert-base-uncased"
model = AutoAdapterModel.from_pretrained(model_name)
adapter_name = model.load_adapter("ner/conll2003@ukp")

model.active_adapters = adapter_name
model.active_head = "ner_head"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# How you can acces the labels and the mapping for a pretrained head
print(model.get_labels())
print(model.get_labels_dict())

['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']
{1: 'B-LOC', 7: 'B-MISC', 5: 'B-ORG', 3: 'B-PER', 2: 'I-LOC', 8: 'I-MISC', 6: 'I-ORG', 4: 'I-PER', 0: 'O'}


This helper function allows us to get the sequence of ids of the predicted output for a specific input sentence.

In [3]:
def predict(sentence):
  tokens = tokenizer.encode(
        sentence,
        return_tensors="pt",
    )
  model.eval()
  preds = model(tokens, adapter_names=['ner'])[0]
  preds = preds.detach().numpy()
  preds = np.argmax(preds, axis=2)
  return tokenizer.tokenize(sentence), preds.squeeze()[1:-1]

If we want to use the model to predict the labels of a sentence we can use the `model.get_labels_dict()` function to map the predicted label ids to the corresponding label, as for the example text.

In [4]:
example_text="Germany's representative to the European Union\'s veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer."
# Get the mapping of ids to labels
label_map = model.get_labels_dict()
tokens, preds = predict(example_text)
for token, pred in zip(tokens, preds):
  print(f"{token}({label_map[pred]}) ", end="")

germany(B-LOC) '(O) s(O) representative(O) to(O) the(O) european(B-ORG) union(I-ORG) '(O) s(O) veterinary(B-ORG) committee(I-ORG) werner(B-PER) z(I-PER) ##wing(I-PER) ##mann(I-PER) said(O) on(O) wednesday(O) consumers(O) should(O) buy(O) sheep(O) ##me(O) ##at(O) from(O) countries(O) other(O) than(O) britain(B-LOC) until(O) the(O) scientific(O) advice(O) was(O) clearer(O) .(O) 